In [75]:
import pandas as pd
from ortools.sat.python import cp_model
import numpy as np

### forecast demand

In [76]:
demand_table = pd.DataFrame({
    'Week' : [1, 2, 3, 4],
    'Demand_1' : [23, 27,34,40],
    'Demand 2' : [11, 13, 15, 14]
})
demand_table

,Week,Demand_1,Demand 2
0,1,23,11
1,2,27,13
2,3,34,15
3,4,40,14


In [77]:
# Mt = µYt + (1- µ)Mt-1
def Mt(mu, Yt, Mt_prev):
    return mu * Yt + (1-mu)*Mt_prev

In [78]:
def forecast_with_exponential_smoothing(df_column, mu):
    forecast = [df_column[0]] # initial forecast is the first value in the column
    for i in range(1, len(df_column)):
        forecast.append(Mt(mu, df_column[i], forecast[i-1])) 
    return forecast

In [79]:
forecast_1 = forecast_with_exponential_smoothing(demand_table['Demand_1'], 0.7)
forecast_1

[23, 25.8, 31.54, 37.462]

In [80]:
forecast_2 = forecast_with_exponential_smoothing(demand_table['Demand 2'], 0.7)
forecast_2

[11, 12.4, 14.22, 14.065999999999999]

In [81]:
predicted_demand_1 = np.round(forecast_1[-1]).astype(int)
predicted_demand_2 = np.round(forecast_2[-1]).astype(int)
print(f'Predicted demand 1: {predicted_demand_1}, Predicted demand 2: {predicted_demand_2}')

Predicted demand 1: 37, Predicted demand 2: 14


### build and solve LP model

In [82]:
model = cp_model.CpModel()

In [83]:
x1 = model.NewIntVar(lb=0, ub=10000, name='Produced amount P1')
x2 = model.NewIntVar(lb=0, ub=10000, name='Produced amount P2')

In [84]:
# constraints

# available machine time
c1 = model.Add(10 * x1 + 4 * x2 <= 1200) # machine X
c2 = model.Add(25 * x1 + 45 * x2 <= 900) # machine Y

# demand 
c3 = model.Add(x1 <= predicted_demand_1)
c4 = model.Add(x2 <= predicted_demand_2)

In [85]:
# objective
model.Maximize(10 * x1 + 4 * x2 - (37 - x1) * 3 - (14 - x2))

In [86]:
# solve model
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [88]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Anzahl P1: ", solver.Value(x1))
    print("Anzahl P2: ", solver.Value(x2))
    print("Maximaler Gewinn: ", solver.ObjectiveValue())
else:
    print("No solution found.")

Anzahl P1:  36
Anzahl P2:  0
Maximaler Gewinn:  343.0
